# Hello World OpenCV

## Imports

In [ ]:
import cv2
from google.colab.patches import cv2_imshow # cv2.imshow doesnt work in Colab
import os

print("OpenCV Version: {}".format(cv2.__version__))

## Get some image

In [ ]:
if os.path.isfile('lena.png'):
    print ("Test Image File exist")
else:
    print ("Test Image File does not exist; downloading...")
    import urllib.request as urllib_request
    urllib_request.urlretrieve("https://upload.wikimedia.org/wikipedia/en/7/7d/Lenna_%28test_image%29.png", "lena.png")

image = cv2.imread("./lena.png")
cv2_imshow(image) # should be: cv2.imshow("Image", image)'
cv2.waitKey(0)
cv2.destroyAllWindows()

## Images are stored in BRG instead of RGB

In [ ]:
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
cv2_imshow(rgb_image) # cv2.imshow("Image RGB", rgb_image)
cv2.waitKey(0)
cv2.destroyAllWindows()


## Define a visualization function

In [ ]:
def viewImage(image, name_of_window):
    # cv2.namedWindow(name_of_window, cv2.WINDOW_AUTOSIZE) # cv2.namedWindow crashes Colab
    cv2_imshow(image) #should be: cv2.imshow(name_of_window, image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
viewImage(image, "Lena")

## Edit pixels

In [ ]:
edited = image.copy()
edited[200:390, 200:360, 0] = 255
viewImage(edited, "Lena edited")


## Cropping

In [ ]:
cropped = image[200:390, 200:360]
viewImage(cropped, "Lena cropped")

## Resizing

In [ ]:
scale_percent = 10 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)
resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
viewImage(resized, "Lena resized to {}%".format(scale_percent))


## Drawing a Rectangle

In [ ]:
output = image.copy()
cv2.rectangle(img=output, pt1=(200, 200), pt2=(360, 390), color=(255, 0, 0), thickness=10)
viewImage(output, "Lena with a rectangle")


## Drawing a line

In [ ]:
cv2.line(img=output, pt1=(256, 390), pt2=(256, 512), color=(0, 0, 255), thickness=5)
viewImage(output, "Lena with a line")

## Writing on an image

In [ ]:
cv2.putText(img=output, text="Lena", org=(360, 390), fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=2, color=(0, 255, 0), thickness=2)
viewImage(output, "Lena with text")

## Saving an image

In [ ]:
cv2.imwrite("./output.jpg", output)
!pwd
!ls

## Blurring/Smoothing

In [ ]:
blurred = cv2.GaussianBlur(src=image, ksize=(15, 15), sigmaX=2)
viewImage(blurred, "Lena blurred")

## Rotating

In [ ]:
(h, w, d) = image.shape
center = (w // 2, h // 2)
rot = 45
rotMat = cv2.getRotationMatrix2D(center=center, angle=rot, scale=1.0)
rotated = cv2.warpAffine(src=image, M=rotMat, dsize=(w, h))
viewImage(rotated, "Lena rotated by {} degrees".format(rot))

## Blend

In [ ]:
alpha_slider_max = 100
weight = 50

def on_trackbar_weight(val):
    alpha = val / alpha_slider_max
    beta = ( 1.0 - alpha )
    blend = cv2.addWeighted(image, alpha, rotated, beta, 0.0)
    #cv2.imshow('Lena blended', blend)
    cv2_imshow(blend)

#cv2.namedWindow('Lena blended')
#trackbar_name = 'Alpha 0 - {}'.format(alpha_slider_max)
#cv2.createTrackbar(trackbar_name, 'Lena blended' , 50, alpha_slider_max, on_trackbar_weight)
on_trackbar_weight(weight)
cv2.waitKey()
#cv2.destroyWindow('Lena blended')


## Grayscaling

In [ ]:
gray_image = cv2.cvtColor(src=image, code=cv2.COLOR_BGR2GRAY)
viewImage(gray_image, "Lena gray-scale")

## Thresholding

In [ ]:
threshold_slider_max = 255
threshold = 200

def on_trackbar_threshold(val):
    threshold = val
    ret, threshold_image = cv2.threshold(gray_image, threshold, 255, 0)
    #cv2.imshow("Lena thresholded", threshold_image)
    cv2_imshow(threshold_image)

# cv2.namedWindow("Lena thresholded")
# trackbar_name = "Threshold 0 - {}".format(threshold_slider_max)
# cv2.createTrackbar(trackbar_name, "Lena thresholded" , threshold, threshold_slider_max, on_trackbar_threshold)
on_trackbar_threshold(threshold)
#cv2.waitKey()
#cv2.destroyWindow("Lena thresholded")

ret, threshold_image = cv2.threshold(src=gray_image, thresh=threshold, maxval=255, type=0)

## Contours

In [ ]:
contours, hierarchy = cv2.findContours(image=threshold_image, mode=cv2.RETR_TREE, method=cv2.CHAIN_APPROX_NONE)
image_with_contours= image.copy()
cv2.drawContours(image=image_with_contours, contours=contours, contourIdx=-1, color=(255, 0, 0), thickness=1)
viewImage(image_with_contours, "Lena contours")


## Face Detection

In [ ]:
if os.path.isfile('haarcascade_frontalface_default.xml'):
    print ("Detection Model File exists")
else:
    print ("Detection Model File does not exist; downloading...")
    import urllib.request as urllib_request
    urllib_request.urlretrieve("https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml", "haarcascade_frontalface_default.xml")

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
faces = face_cascade.detectMultiScale(gray_image)
print("Lena with {} faces detected".format(len(faces)))
image_faces= image.copy()
for (x, y, w, h) in faces:
    cv2.rectangle(image_faces, (x, y), (x+w, y+h), (0, 255, 0), 2)
viewImage(image_faces,"Lena with {} faces detected".format(len(faces)))


## Cleanup

In [ ]:
!pwd
!ls -la
!rm lena.png
!rm output.jpg
!rm haarcascade_frontalface_default.xml
!ls -la